In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='Groq_api_key', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
response.content

'The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.'

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-44297?from=job%20search%20funnel")
web_page = loader.load().pop().page_content
print(web_page)

Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':web_page})
print(res.content)

```json
{
  "role": "Lead Software Engineer, ITC",
  "experience": "8+ years of hands-on industry software development experience",
  "skills": [
    "Experience with front-end frameworks like React, Angular, or Vue.js",
    "Experience with cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools",
    "Experience with AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3",
    "Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest",
    "Strong understanding of architectural design patterns and computer-science fundamentals",
    "Experience with implementing and integrating AI, Machine Learning and related data solutions",
    "Hands-on experience implementing and supporting modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)"
  ],
  "description": "We’re looking for a Lead Software Engineer to solve complex software engineering problems supporting Nike’s pursuit o

In [15]:
type(res.content)

str

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years of hands-on industry software development experience',
 'skills': ['Experience with front-end frameworks like React, Angular, or Vue.js',
  'Experience with cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
  'Experience with AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
  'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
  'Strong understanding of architectural design patterns and computer-science fundamentals',
  'Experience with implementing and integrating AI, Machine Learning and related data solutions',
  'Hands-on experience implementing and supporting modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)'],
 'description': 'We’re looking for a Lead Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tool

In [18]:
type(json_res)

dict

In [20]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [21]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [24]:
links = collection.query(
    query_texts=["Experience in python", "Experience in React"],
    n_results=2 # how many results to return
).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [25]:
my_job = json_res
my_job["skills"]

['Experience with front-end frameworks like React, Angular, or Vue.js',
 'Experience with cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'Experience with AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'Strong understanding of architectural design patterns and computer-science fundamentals',
 'Experience with implementing and integrating AI, Machine Learning and related data solutions',
 'Hands-on experience implementing and supporting modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)']

In [26]:
links = collection.query(
    query_texts=my_job["skills"],
    n_results=2 # how many results to return
).get('metadatas',[])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [27]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(my_job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's ITC Needs

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you solve complex software engineering problems. Our team of experts has a proven track record of delivering state-of-the-art solutions that drive business growth and efficiency.

With over 8 years of experience in native cloud software engineering, our team is well-equipped to handle the requirements mentioned in the job description. We have hands-on experience with front-end frameworks like React, Angular, and Vue.js, as well as expertise in cloud architecture, modern DevOps, infrastructure as code, CI/CD, and related tools. Our team is also familiar with AWS products, including Lambda, Step Functions, DynamoDB, Elasticsearch, and S3.

In addition, we have a strong understanding of architectural design patterns and computer-science fu